# Лабораторная работа № 4. Модель однофазной многоканальной замкнутой системы обслуживания
Выполнил Пакало Александр Сергеевич, студент РТ5-81Б

Вариант № 5

## Задание № 1
Вычислительный центр фирмы состоит из $m$
 главных серверов коллективного пользования. Число работающих в центре
 программистов в любой момент времени равно
$k$. Каждый программист готовит свою программу и через терминал передает ее
на сервер для выполнения, куда она сразу попадает. Время подготовки программ
имеет экспоненциальное распределение со средним значением
$t_1$ мин. Время выполнения программы на любом из серверов имеет экспоненциальное
 распределение со средним значением
 $t_2$ мин. Каждый программист ожидает ответа от сервера, прежде, чем начнет писать следующую программу.

Найти (теоретически и экспериментально):
1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал.
2. Среднее время до получения пользователем результатов реализации.
3. Среднее количество программ, ожидающих выполнения на сервере.

Требуется выполнить расчет при заданном по варианту значении
$m$, а также при $m = 1$.

In [20]:
Variant <- 5
set.seed(Variant)
k <- sample(c(10:25), 1)
m <- sample(c(3:6), 1)
t1 <- sample(c(14:20), 1)
t2 <- sample(c(2:6), 1)
View(data.frame(k, m, t1, t2))

k,m,t1,t2
<int>,<int>,<int>,<int>
11,5,14,4


Найдем интенсивность выполнения программ:

In [21]:
lambda_1 <- 1 / t1
lambda_1

[1] 0.07142857

Интенсивность поступления программ от $k$ программистов:

In [22]:
lambda_k <- k * lambda_1
lambda_k

[1] 0.7857143

Интенсивность выполнения программ:

In [23]:
miu <- 1 / t2
miu

[1] 0.25

Интенсивность выполнения программ меньше интенсивности поступления,
следовательно очередь в нашей системе будет бесконечно расти. Согласно
указаниям преподавателя, изменим начальные параметры так, чтобы неравенство
обратилось:

In [202]:
t1 <- 45
lambda_k <- k / t1
lambda_k

[1] 0.2444444

Итого имеем следующие начальные величины:

In [203]:
View(data.frame(k, m, t1, t2))

k,m,t1,t2
<int>,<int>,<dbl>,<int>
11,5,45,4


In [ ]:
# Зададим таблицу результатов.

In [204]:
results <- data.frame(probability = c("-", "-"), T = c("-", "-"), Lqueue = c("-", "-"))
row.names(results) <- c('theoretical', 'practical')
results

,probability,T,Lqueue
,<chr>,<chr>,<chr>
theoretical,-,-,-
practical,-,-,-


### Теоретически

#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал
Мы имеем дело с многоканальной **замкнутой** СМО с неограниченной очередью
$M / M / m / \infty$. Следовательно:

$$
P_0 = \left(
  \sum_{i = 0}^m \frac{k! \rho^i}{\left(k - i \right)!i!}
  +
  \sum_{i = 1}^{k - m} \frac{k! \rho^{m + i}}{m! \cdot (k - m - i)!m^i}
\right) ^ {-1}
$$
где $\rho = \frac{\lambda}{\mu}$

In [205]:
ro <- lambda_1 / miu
ro

[1] 0.2857143

In [206]:
i1 <- 0:m
parts_of_sum1 <- c(factorial(k) * ro^i1 / (factorial((k - i1)) * factorial(i1)))

i2 <- 1:(k - m)
parts_of_sum2 <- c(factorial(k) * ro^(m + i2) / (factorial(m) * factorial((k - m - i2)) * m^i2))

P0 <- (sum(parts_of_sum1) + sum(parts_of_sum2))^(-1)
P0

[1] 0.06261347

In [207]:
P0 * k * ro

[1] 0.1967852

Тогда вероятность того, что $i$ серверов занято:
$$
P_i = P_0 \cdot \frac{k! \rho^i}{\left(k - i \right)!i!}
$$

Для этого воспользуемся уже созданным массивом частей суммы:

In [208]:
get_Pi <- function(part) part * P0

In [209]:
Pi <- lapply(parts_of_sum1[1:m], get_Pi)
Pi

[[1]]
[1] 0.06261347

[[2]]
[1] 0.1967852

[[3]]
[1] 0.2811217

[[4]]
[1] 0.2409615

[[5]]
[1] 0.1376923

Тогда $\sum_{i = 0}^{m} P_i$ -  вероятность того, что хотя бы один сервер
будет доступен. Следовательно, обратная вероятность и будет нашим ответом.

In [220]:
probability <- 1 - sum(unlist(Pi))
results$probability[1] <- probability
probability

[1] 0.08082587

#### 2. Среднее время до получения пользователем результатов реализации.
По формуле Литтла, подходящей для СМО любого вида имеем:
$$
T_{\text{сист}} = \frac{L_{\text{сист}}}{\Lambda} \\
\Lambda = (k - L_{\text{сист}}) \cdot \lambda \\
L_{\text{сист}} = \sum_{i = 1}^k \left(P_i \cdot i \right)
$$

In [211]:
Pi_queue <- mapply(function(Pi, i) Pi * i, Pi[1:k], 1:k)
Lsys <- sum(unlist(Pi))
Lsys

[1] 0.9191741

In [212]:
Lambda <- (k - Lsys) * lambda_1
Lambda

[1] 0.720059

In [213]:
T <- Lsys / Lambda
results$T[1] <- T
T

[1] 1.276526

#### 3. Среднее количество программ, ожидающих выполнения на сервере.
Она же средняя длина очереди:
$$
L_{\text{оч}} = \frac{\rho^{m+1}}{m! \cdot m} \cdot P_0 \cdot
\frac{1}{\left( 1 - \frac{\rho}{m} \right)^2}
$$

In [214]:
Lqueue <- ro^(m + 1) / (factorial(m) * m) * P0 * 1 / (1 - ro / m)^2
results$Lqueue[1] <- Lqueue
Lqueue

[1] 6.385817e-08

In [215]:
results

,probability,T,Lqueue
,<chr>,<chr>,<chr>
theoretical,0.0808258656604589,1.27652615492436,6.38581729495202e-08
practical,-,-,-


Как видно, очередь система работает достаточно быстро, чтобы не накапливать очередь.
Это подтверждается и при сравнении интенсивностей поступления / обработки программ.

### Численно

In [261]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

SIMULATION_TIME <- 10000

env <- simmer("SupaDupaSim")

limit_programs_in_system <- trajectory() %>%
    leave(
        function() as.integer(
            get_queue_count(env, "server") + get_seized(env, "server") >= k
        )
    )

programs <- trajectory() %>%
    # join(limit_programs_in_system) %>%
    seize("server", 1) %>%
    timeout(function() rexp(1, 1 / t1)) %>%
    release("server", 1)

env %>%
    add_resource("server", capacity = m, queue_size = k - m) %>%
    add_generator(
        "programs",
        programs,
        distribution = function() rexp(1, lambda_k)
    ) %>%
    run(until = SIMULATION_TIME)

simmer environment: SupaDupaSim | now: 10000 | next: 10002.8850006089
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 5(5) | queue status: 6(6) }
{ Source: programs | monitored: 1 | n_generated: 2423 }

#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал.

In [262]:
resources <- get_mon_resources(env)
resources

resource,time,server,queue,capacity,queue_size,system,limit,replication
<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
server,16.67226,1,0,5,6,1,11,1
server,31.24034,2,0,5,6,2,11,1
server,35.81829,3,0,5,6,3,11,1
server,36.64213,2,0,5,6,2,11,1
server,40.92799,3,0,5,6,3,11,1
server,51.37941,4,0,5,6,4,11,1
server,57.45350,5,0,5,6,5,11,1
server,57.73664,4,0,5,6,4,11,1
server,59.29181,3,0,5,6,3,11,1


In [264]:
probability <- resources %>% with(sum(server == m) / length(server)) / 2
results$probability[2] <- probability

#### 2. Среднее время до получения пользователем результатов реализации.

In [265]:
arrivals <- get_mon_arrivals(env)
arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs2,35.81829,36.64213,0.8238416,TRUE,1
programs0,16.67226,57.73664,41.0643827,TRUE,1
programs4,51.37941,59.29181,7.9124009,TRUE,1
programs14,80.04084,80.04084,0.0000000,FALSE,1
programs15,81.16490,81.16490,0.0000000,FALSE,1
programs16,95.57392,95.57392,0.0000000,FALSE,1
programs5,57.45350,97.41570,39.9622043,TRUE,1
programs1,31.24034,100.95869,69.7183571,TRUE,1
programs7,63.68863,102.75857,39.0699411,TRUE,1


In [266]:
T <-arrivals %>% subset(finished) %>% with(mean(end_time - start_time))
results$T[2] <- T

#### 3. Среднее количество программ, ожидающих выполнения на сервере.

In [272]:
Lqueue <- resources %>% with(mean(queue))
results$Lqueue[2] <- Lqueue

[1] 4.71037

In [277]:
results

,probability,T,Lqueue
,<chr>,<chr>,<chr>
theoretical,0.0808258656604589,1.27652615492436,6.38581729495202e-08
practical,0.0921233708490012,1.24012378913035,7.38192123038837e-08
